In [9]:
import csv
from itertools import combinations

class Player:
    def __init__(self, pid, name, rating=0):
        self.id = pid
        self.name = name
        self.rating = rating
        self.points = 0
        self.wins = 0
        self.losses = 0
        self.draws = 0
        self.opponents = []   # track who they played
        self.sb_score = 0
        self.buchholz = 0

    def __repr__(self):
        return f"{self.name}({self.points} pts)"


In [10]:
def round_robin(players, round_no):
    n = len(players)
    if n % 2 == 1:
        players.append(Player(-1, "BYE"))
        n += 1
    idx = list(range(n))
    for r in range(round_no):
        round_pairs = []
        for i in range(n // 2):
            p1, p2 = idx[i], idx[n - 1 - i]
            if players[p1].id != -1 and players[p2].id != -1:
                round_pairs.append((players[p1], players[p2]))
        idx = [idx[0]] + idx[-1:] + idx[1:-1]
    return round_pairs

def swiss_system(players, round_no):
    players.sort(key=lambda p: (-p.points, -p.rating))
    used = set()
    pairings = []
    for i in range(0, len(players)-1, 2):
        if i not in used and (i+1) not in used:
            p1, p2 = players[i], players[i+1]
            pairings.append((p1, p2))
            used.add(i); used.add(i+1)
    return pairings

def knockout(players, round_no):
    players.sort(key=lambda p: -p.rating)
    return [(players[i], players[i+1]) for i in range(0, len(players)-1, 2)]


In [11]:
def update_tiebreaks(players):
    # Buchholz = sum of opponents’ points
    for p in players:
        p.buchholz = sum(opp.points for opp in p.opponents)

    # Sonneborn–Berger = sum(opponent points) if win, + half if draw
    for p in players:
        sb = 0
        for opp in p.opponents:
            if p.points > opp.points:  # won
                sb += opp.points
            elif p.points == opp.points and p.draws > 0:
                sb += opp.points / 2
        p.sb_score = sb


In [12]:
def run_tournament(players, system, round_no):
    if system == "roundrobin":
        pairings = round_robin(players, round_no)
    elif system == "swiss":
        pairings = swiss_system(players, round_no)
    elif system == "knockout":
        pairings = knockout(players, round_no)
    else:
        print("Unknown system:", system)
        return
    
    print(f"\n🎯 {system.capitalize()} Round {round_no} Pairings:")
    for p1, p2 in pairings:
        print(f"{p1.name}  vs  {p2.name}")

    show_standings(players)
    return pairings


In [13]:
def show_standings(players):
    update_tiebreaks(players)
    players.sort(key=lambda p: (-p.points, -p.sb_score, -p.buchholz, -p.rating))
    print("🏆 Standings:")
    print("ID | Name                | Rating | Pts | SB   | Buchholz")
    print("-"*65)
    for p in players:
        print(f"{p.id:2} | {p.name:18} | {p.rating:6} | {p.points:3} | {p.sb_score:4.1f} | {p.buchholz:4.1f}")


In [14]:
def enter_results(pairings):
    for p1, p2 in pairings:
        print(f"Result for {p1.name} vs {p2.name} (1=win p1, 0=win p2, 0.5=draw): ", end="")
        res = input().strip()
        if res == "1":
            p1.points += 1; p1.wins += 1; p2.losses += 1
        elif res == "0":
            p2.points += 1; p2.wins += 1; p1.losses += 1
        else:  # draw
            p1.points += 0.5; p2.points += 0.5
            p1.draws += 1; p2.draws += 1
        p1.opponents.append(p2)
        p2.opponents.append(p1)


In [15]:
def run_tournament(players, system, total_rounds):
    for r in range(1, total_rounds+1):
        if system == "roundrobin":
            pairings = round_robin(players, r)
        elif system == "swiss":
            pairings = swiss_system(players, r)
        elif system == "knockout":
            pairings = knockout(players, r)
        else:
            print("Unknown system"); return

        print(f"\n🎯 {system.capitalize()} Round {r} Pairings:")
        for p1, p2 in pairings:
            print(f"{p1.name} vs {p2.name}")

        enter_results(pairings)
        show_standings(players)


In [16]:
players = [
    Player(1, "Magnus Carlsen", 2865),
    Player(2, "Ian Nepomniachtchi", 2795),
    Player(3, "Ding Liren", 2780),
    Player(4, "Fabiano Caruana", 2782),
    Player(5, "Hikaru Nakamura", 2775),
    Player(6, "Alireza Firouzja", 2760),
]


In [ ]:
# Choose: "swiss", "roundrobin", or "knockout"
run_tournament(players, "swiss", total_rounds=3)



🎯 Swiss Round 1 Pairings:
Magnus Carlsen vs Ian Nepomniachtchi
Fabiano Caruana vs Ding Liren
Hikaru Nakamura vs Alireza Firouzja
Result for Magnus Carlsen vs Ian Nepomniachtchi (1=win p1, 0=win p2, 0.5=draw): Result for Fabiano Caruana vs Ding Liren (1=win p1, 0=win p2, 0.5=draw): 